# Feature Engineering and Modelling

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [6]:
# Load dataset
data = pd.read_csv('../data/raw/clean_data_after_eda.csv', parse_dates=['date_activ', 'date_end', 'date_modif_prod', 'date_renewal'])

# Drop unnecessary columns
data.drop(columns=['id'], inplace=True)

# Extract year and month from dates
data['year_activ'] = data['date_activ'].dt.year
data['month_activ'] = data['date_activ'].dt.month
data['year_end'] = data['date_end'].dt.year
data['month_end'] = data['date_end'].dt.month
data['tenure_days'] = (data['date_end'] - data['date_activ']).dt.days

# Compute off-peak price difference (December - January of preceding year)
data['price_diff_dec_jan'] = data['var_6m_price_off_peak'].shift(1) - data['var_6m_price_off_peak'].shift(12)

# Fill missing values
data.fillna(0, inplace=True)

# Encode categorical variables
data = pd.get_dummies(data, columns=['channel_sales'], drop_first=True)

In [7]:
# Define features and target variable
X = data.drop(columns=['churn'])
y = data['churn']

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

TypeError: Cannot cast DatetimeArray to dtype float64

In [2]:
# Feature Importance Analysis
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': rf_model.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Plot Feature Importances
plt.figure(figsize=(10,6))
sns.barplot(x='Importance', y='Feature', data=feature_importances.head(15), palette='viridis')
plt.title('Top 15 Feature Importances')
plt.show()

# Hyperparameter Tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best Parameters
print("Best Parameters:", grid_search.best_params_)

# Train best model
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

# Evaluate best model
best_accuracy = accuracy_score(y_test, y_pred_best)
print(f"Best Model Accuracy: {best_accuracy:.4f}")